In [1]:
from confluent_kafka import Producer, KafkaError
import json
import ccloud_lib

In [ ]:
config_file = "python.config"
topic = "first-topic"
conf = ccloud_lib.read_ccloud_config(config_file)
conf

In [ ]:
# Create Producer instance
producer_conf = ccloud_lib.pop_schema_registry_params_from_config(conf)
producer_conf

In [5]:
producer = Producer(producer_conf)

%4|1658772576.611|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance


In [6]:
topic = "sparsh-first-topic"

In [7]:
# Create topic if needed
ccloud_lib.create_topic(conf, topic)

%4|1658772665.240|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance


Topic sparsh-first-topic created


In [8]:
for n in range(10):
    record_key = "alice"
    record_value = json.dumps({'count': n})
    print("Producing record: {}\t{}".format(record_key, record_value))
    producer.produce(topic, key=record_key, value=record_value, on_delivery=ccloud_lib.acked)
    producer.poll(0)

producer.flush()

Producing record: alice	{"count": 0}
Producing record: alice	{"count": 1}
Producing record: alice	{"count": 2}
Producing record: alice	{"count": 3}
Producing record: alice	{"count": 4}
Producing record: alice	{"count": 5}
Producing record: alice	{"count": 6}
Producing record: alice	{"count": 7}
Producing record: alice	{"count": 8}
Producing record: alice	{"count": 9}
Produced record to topic sparsh-first-topic partition [0] @ offset 0
Produced record to topic sparsh-first-topic partition [0] @ offset 1
Produced record to topic sparsh-first-topic partition [0] @ offset 2
Produced record to topic sparsh-first-topic partition [0] @ offset 3
Produced record to topic sparsh-first-topic partition [0] @ offset 4
Produced record to topic sparsh-first-topic partition [0] @ offset 5
Produced record to topic sparsh-first-topic partition [0] @ offset 6
Produced record to topic sparsh-first-topic partition [0] @ offset 7
Produced record to topic sparsh-first-topic partition [0] @ offset 8
Produced 

0

In [9]:
from confluent_kafka import Consumer

In [10]:
consumer_conf = ccloud_lib.pop_schema_registry_params_from_config(conf)
consumer_conf['group.id'] = 'python_example_group_1'
consumer_conf['auto.offset.reset'] = 'earliest'
consumer = Consumer(consumer_conf)

In [11]:
topics = ["sparsh-first-topic", "first-topic"]

In [12]:
# Subscribe to topic
consumer.subscribe(topics)

In [13]:
# Process messages
total_count = 0
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            # No message available within timeout.
            # Initial message consumption may take up to
            # `session.timeout.ms` for the consumer group to
            # rebalance and start consuming
            print("Waiting for message or event/error in poll()")
            continue
        elif msg.error():
            print('error: {}'.format(msg.error()))
        else:
            # Check for Kafka message
            record_key = msg.key()
            record_value = msg.value()
            data = json.loads(record_value)
            count = data['count']
            total_count += count
            print("Consumed record with key {} and value {}, \
                    and updated total count to {}"
                    .format(record_key, record_value, total_count))
except KeyboardInterrupt:
    pass
finally:
    # Leave group and commit final offsets
    consumer.close()

Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Waiting for message or event/error in poll()
Consumed record with key b'alice' and value b'{"count": 0}',                     and updated total count to 0
Consumed record with key b'alice' and value b'{"count": 1}',                     and updated total count to 1
Consumed record with key b'alice' and value b'{"count": 2}',                     and updated total count to 3
Consumed record with key b'alice' and value b'{"count": 3}',                     and updated total count to 6
Consumed record with key b'alice' and value b'{"count": 4}',                     and updated total count to 10
Consumed record with key b'alice' and value b'{"count": 5}',                     and updated total count to 15
Consumed record with key b'alice' and value b'{"count": 6}',                     and updated total count to 21
Consumed record with key b'alice' and value b'{"count": 7}',                     and updated

In [5]:
!./producer.py -f python.config -t first-topic

%4|1657920679.098|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance
%4|1657920679.112|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance
Producing record: alice	{"count": 0}
Producing record: alice	{"count": 1}
Producing record: alice	{"count": 2}
Producing record: alice	{"count": 3}
Producing record: alice	{"count": 4}
Producing record: alice	{"count": 5}
Producing record: alice	{"count": 6}
Producing record: alice	{"count": 7}
Producing record: alice	{"count": 8}
Producing record: alice	{"count": 9}
Produced record to topic test1 partition [0] @ offset 10
Produced record to topic test1 partition [0] @ offset 11
Produced record to topic test1 partition [0] @ offset 12
Produced record to topic test1 partition [0] @ offset 13
Produced record to topic test1 partition [0] @ offset 14
Produ

In [ ]:
!./consumer.py -f python.config -t test1

In [6]:
# verify schema registry subjects
!curl -u <>:U/<>/<> https://<>.us-east-2.aws.confluent.cloud/subjects

[]

In [8]:
!./producer_ccsr.py -f python.config -t test2

%4|1657921054.754|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance
%4|1657921059.448|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance
Producing Avro record: alice	0
Producing Avro record: alice	1
Producing Avro record: alice	2
Producing Avro record: alice	3
Producing Avro record: alice	4
Producing Avro record: alice	5
Producing Avro record: alice	6
Producing Avro record: alice	7
Producing Avro record: alice	8
Producing Avro record: alice	9
Produced record to topic test2 partition [0] @ offset 10
Produced record to topic test2 partition [0] @ offset 11
Produced record to topic test2 partition [0] @ offset 12
Produced record to topic test2 partition [0] @ offset 13
Produced record to topic test2 partition [0] @ offset 14
Produced record to topic test2 partition [0] @ offset 15
Produced

In [ ]:
!./consumer_ccsr.py -f python.config -t test2

In [10]:
# verify schema information for subject
!curl -u <>:U/<>/<> https://psrc-l7y22.us-east-2.aws.confluent.cloud/subjects/test2-value/versions/1

{"subject":"test2-value","version":1,"id":100002,"schema":"{\"type\":\"record\",\"name\":\"Count\",\"namespace\":\"io.confluent.examples.clients.cloud\",\"fields\":[{\"name\":\"count\",\"type\":\"int\"}]}"}